In [ ]:
import numpy as np
pi = np.pi
import matplotlib.pyplot as plt
import scipy as sp
from scipy import fftpack, signal
import time

# À la découverte de la transformée de Fourier à court terme

Dans le cours, on a vu que la transformée de Fourier $X : \nu \mapsto X(\nu)$ d'un signal $x : t \mapsto x(t)$ est une fonction complexe. Il y a donc deux manières de l'analyser : soit en découplant partie réelle $\mathfrak{R}_e(X(\nu))$ et partie imaginaire $\mathfrak{I}_m(X(\nu))$, soit en découplant module $\vert X(\nu) \vert$ et phase $\phi(X(\nu))$. En pratique, c'est la deuxième approche qui est privilégiée puisque plus facilement interprétable.

En particulier, le module $\vert X(\nu ) \vert$, appelé **spectre** de $x$ permet de visualiser la distribution d'amplitude des fréquences contenues dans le signal $x$ : un signal "basse fréquence" aura un spectre majoritairement concentré autour de la fréquence nulle $\nu = 0$. La phase, quant à elle, encode la position spatiale des différentes fréquences dans le signal $x$. Si la phase n'est visuellement d'aucune aide pour l'interprétation du contenu fréquentiel de $x$, elle est néanmoins porteuse d'une information primordiale : en effet, deux signaux $x : t \mapsto x(t)$ et $x^- : t \mapsto x^-(t) = x(-t)$ ont le même spectre (même contenu fréquentiel) mais pas la même phase (puisque les fréquences ne sont pas situées au même endroit dans le signal).

In [ ]:
tmin = 0
tmax = 1
Te = 0.0001 # 10000 points/seconde
fe = 1/Te
t = np.arange(tmin,tmax,Te) 

On définit tout d'abord un premier signal $s_1$ comme une superposition de 3 sinus de fréquences 5 Hz, 10 Hz et 15 Hz, chacun d'amplitude 1 (autrement dit, $s_1$ est la combinaison d'un sinus de fréquence fondamentale 5 Hz et de ses deux premiers harmoniques)

In [ ]:
f1 = 5 
T1 = 1/f1
s1 = 1*np.sin(2*pi*f1*t)+1*np.sin(2*pi*(2*f1)*t)+1*np.sin(2*pi*(3*f1)*t)
# ----------------
plt.figure(figsize=(8,5))
plt.plot(t,s1,'b',label=r'$s_1(t)$')
plt.xlim(t.min(),t.max())
plt.legend(loc='upper right',fontsize=16)
plt.show()

<font color="blue"> **Question** Quelle est donc la période fondamentale de $s_1$ ? Autrement dit, combien de périodes observe t-on sur une durée d'une seconde ? </font>

In [ ]:
# TODO

On définit ensuite un deuxième signal $s_2$ comme une superposition de 3 sinus de fréquences 20 Hz, 40 Hz et 60 Hz, chacun d'amplitude 1 (autrement dit, $s_2$ est la combinaison d'un sinus de fréquence fondamentale 20 Hz et de ses deux premiers harmoniques)

In [ ]:
f2 = 20
T2 = 1/f2
s2 = 1*np.sin(2*pi*f2*t)+1*np.sin(2*pi*(2*f2)*t)+1*np.sin(2*pi*(3*f2)*t)
# ----------------
plt.figure(figsize=(8,5))
plt.plot(t,s2,'b',label=r'$s_2(t)$')
plt.xlim(t.min(),t.max())
plt.legend(loc='upper right',fontsize=16)
plt.show()

<font color="blue"> **Question** Même question que précédemment, mais pour $s_2$ : quelle est sa période fondamentale ? Combien de périodes observe t-on sur une durée d'une seconde ? </font>

In [ ]:
# TODO

On définit à partir de $s_1$ et de $s_2$ les trois signaux suivants :
* $x_1(t) = s_1(t) + s_2(t)$ pour $0 \leq t  \leq 1s$
* $x_2(t) = \left\lbrace \begin{array}{ll} s_1(t) & \text{ si } t < 0.4s\\ s_2(t) & \text{ si } t \geq 0.4s\end{array}\right.$
* $x_3(t) = \left\lbrace \begin{array}{ll} s_2(t) & \text{ si } t < 0.4s\\
                                           s_1(t) + s_2(t) & \text{ si } 0.4s \leq t < 0.6s\\
                                           s_1(t) & \text{ si } t \geq 0.6s\end{array}\right.$

In [ ]:
# --- x1 ---
x1 = s1+s2
# --- x2---
x2 = np.zeros(t.size)
x2[t<2*T1] = s1[t<2*T1]
x2[t>=2*T1] = s2[t>=2*T1]
# --- x3 ---
x3 = np.zeros(t.size)
x3[t<2*T1] = s2[t<2*T1]
x3[(t>=2*T1)&(t<3*T1)] = s1[(t>=2*T1)&(t<3*T1)] + s2[(t>=2*T1)&(t<3*T1)]
x3[t>=3*T1] = s1[t>=3*T1]
# --- plot plot plot ---
plt.figure(figsize=(18,5))
plt.subplot(131)
plt.plot(t,x1,'b',label=r'$x_1(t)$')
plt.xlim(t.min(),t.max())
plt.legend(loc='upper right',fontsize=16)
plt.subplot(132)
plt.plot(t,x2,'b',label=r'$x_1(t)$')
plt.xlim(t.min(),t.max())
plt.legend(loc='upper right',fontsize=16)
plt.subplot(133)
plt.plot(t,x3,'b',label=r'$x_3(t)$')
plt.xlim(t.min(),t.max())
plt.legend(loc='upper right',fontsize=16)
plt.show()

<font color="blue"> **Question** Calculez les transformées de Fourier $X_1, X_2$ et $X_3$ des signaux $x_1, x_2$ et $x_3$ et affichez les spectres. Que remarquez-vous ? Est-ce conforme à ce que vous vous attendiez ?</font>

<u> Rappels</u> : Pour calculer et afficher une transformée de Fourier dans les règles de l'art, vous pouvez vous servir de
* [`sp.fftpack.fftfreq`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.fftpack.fftfreq.html) pour récupérer les fréquences discrètes
* [`sp.fftpack.fft`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.fftpack.fft.html#scipy.fftpack.fft) pour calculer la transformée de Fourier discrète (TFD)
* [`sp.fftpack.fftshift`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.fftpack.fftshift.html#scipy.fftpack.fftshift) pour ramener la fréquence nulle au milieu du vecteur de fréquence discrètes et de la TFD pour en faciliter l'affichage

In [ ]:
# TODO

L'exemple précédent illustre le problème de la transformée de Fourier : analyser le spectre ne donne aucune information sur la position des différentes fréquences dans le signal. Différents signaux avec le même contenu fréquentiel (mais pas à la même position) ont donc des transformées de Fourier avec des spectres identiques en théorie (ou quasiment identiques en pratique à cause du mécanisme de calcul de la TFD).

### Une (très brève) introduction à l'analyse temps-fréquence

Pour identifier la position des fréquences dans un signal donné, une solution est d'analyser le signal **localement** plutôt que dans sa totalité.<br>
Le signal $x$ est divisé en plusieurs sous-signaux $x_i, i=1,\dots,M$ (techniquement, il serait plus correct de dire que l'intervalle de temps $[0,T]$ sur lequel $x$ est défini est divisé en $M$ sous-intervalles $[t_i^{\text{début}},t_i^{\text{fin}}]$ qui peuvent se chevaucher ($t_{i+1}^{\text{début}} < t_i^{\text{fin}}$) ou non ($t_{i+1}^{\text{début}} =t_i^{\text{fin}}$)), et les différentes transformées de Fourier $X_i$ sont calculées.<br>
Chaque $X_i$ (et les fréquences qu'il contient) est donc associée à une portion temporelle **locale** de $x$. Cette stratégie, appellée **transformée de Fourier à court terme** (*STFT* pour *short-time Fourier transform*) permet donc d'avoir un accès à la position temporelle des fréquence du signal sans avoir besoin d'analyser la phase 🥳

<img src = "figs/STFT-overview.png" width=600/>

Pour visualiser les résultats de la STFT, il est coutûme de combiner les différents spectres dans un plan temps-fréquence et d'afficher le résultat sous la forme d'une image, appelée **spectrogramme**. Interpréter un spectrogramme demande un peu d'habitude : le temps se lit en abscisse, la fréquence en ordonnées, et le code couleur (ou l'intensité en niveau de gris) correspond à l'amplitude d'une fréquence donnée à un instant donné

<img src = "figs/STFT-spectrogram.png" width=750/>

<u>Préambule</u> : la transformée de Fourier discrète d'un signal de longueur $N$ est également de longueur $N$ (cf exercice de prise en main de la TFD). Autrement dit, plus un signal est "long" ($N$ grand), plus la TFD est calculée sur un grand nombre de points, et la résolution fréquentielle est bonne.

Un dernier point, et non des moindres, à avoir en tête lorsqu'on se lance dans l'analyse temps-fréquence d'un signal est que la résolution fréquentielle est inversement proportionnelle à la résolution temporelle :
* lorsqu'on calcule la transformée de Fourier d'un signal dans son intégralité, on obtient une excellente résolution fréquentielle (puisque $N$ grand), mais on n'est incapable de localiser temporellement les différentes fréquences dans le signal (cf début de l'exercice en cours...)
* si on subdivise le signal en 2 morcreaux (début-moitié et moitié-fin), on pourra donc localiser les fréquences du signal soit dans la 1ère partie, soit dans la deuxième. La résolution temporelle est donc meilleure. Par contre, chaque moitié de signal n'étant plus que de longueur $N/2$, la résolution fréquentielle est donc deux fois plus faible
* plus on subdivise le signal en petits morceaux, et plus on sera capable de localiser les fréquences avec une bonne résolution temporelle. Mais la résolution fréquentielle diminuera d'autant puisque les morceaux de signaux sont de plus en plus courts.

Ce résultat est analogue à l'inégalité d'Heisenberg en physique quantique (qui dit qu'il est impossible de mesurer aussi précisément que l'on veut à la fois la position et la vitesse d'une particule dans l'espace). Pour l'anayse temps-fréquence, il porte même le nom d'*inégalité d'Heisenberg-Gabor*


Le code suivant implémente pour vous (vous n'avez pas besoin de le comprendre pour vous en servir) :
* le calcul de la transformée de Fourier à court terme (fonction `STFT`) qui est une sur-couche de [`sp.signal.stft`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.stft.html#scipy.signal.stft) pour vous faciliter la vie avec les différentes options possibles
* l'affichage des sous-signaux (fonction `plot_STFT`) et leurs transformées de Fourier respectives
* le spectrogramme global (fonction `plot_spectrogram`)

In [ ]:
def STFT(x,Te,nb_sub,window='boxcar'):
    # Calcul de la STFT pour nb_sub subdivisions (fenêtrage classique, pas d'overlap)
    N = x.size # longueur du signal
    fe = 1/Te # fréquence d'échantillonnage du signal
    Nsub = int(N/nb_sub) # longueur de chaque subdivision
    freq_stft,t_stft,X_stft = sp.signal.stft(x,fs=fe,window=window,nperseg=Nsub,noverlap=0,nfft=N,boundary=None)
    t_stft = t_stft[t_stft<=N*Te]
    return freq_stft,t_stft,np.abs(X_stft[:,:t_stft.size])

In [ ]:
def plot_STFT(x,t,Te,nb_sub,numax=False):
    # Affiche les nb_sub sous-signaux de x et leur TFD
    N = x.size
    t_ind = np.linspace(0,N,nb_sub+1,dtype=int)
    nu,t_sub,X_stft = STFT(x,Te,nb_sub) # calcul de la STFT
    nu_ind = np.ones(nu.shape,dtype=bool)
    if numax:
        nu_ind = nu<=numax
    # affichage des signaux et TFD individuelles
    figheight = 10*nb_sub
    plt.figure(figsize=(30,figheight))
    for ii in range(nb_sub):
        plt.subplot(nb_sub,2,ii*2+1)
        plt.plot(t,x,color='gray')
        plt.plot(t[t_ind[ii]:t_ind[ii+1]],x[t_ind[ii]:t_ind[ii+1]],color='blue')
        plt.xlim(t.min(),t.max())
        plt.subplot(nb_sub,2,2*(ii+1))
        plt.plot(nu[nu_ind],X_stft[nu_ind,ii],color='red')
        plt.xlim(nu[nu_ind].min(),nu[nu_ind].max())
    plt.show()

In [ ]:
def plot_spectrogram(XSTFT,numax=False):
    # affiche le spectrogramme de la STFT passée en argument
    nu,t,X_stft = XSTFT
    nu_ind = np.ones(nu.shape,dtype=bool)
    if numax:
        nu_ind = nu<=numax
    # affichage du spectrogramme
    plt.pcolormesh(t,nu[nu_ind],X_stft[nu_ind,:])
    plt.title('Spectrogramme',fontsize=16)
    plt.xlabel('temps (en seconde)',fontsize=16)
    plt.ylabel('fréquence (en cycle/seconde)',fontsize=16)
    plt.yticks(ticks=[5,10,15,20,40,60],labels=['5','10','15','20','40','60'])
    plt.show()

<font color="blue"> **Question** À vous de prendre en main tout ça ! Regardez les transformées de Fourier à court terme pour les trois signaux $x_1, x_2, x_3$ en changeant le nombre de subdivisions. Quelle configuration (nombre de subdivisons) vous permet d'identifier le plus précisément possible le contenu fréquentiel et la localisation temporelle de ces fréquences pour les 3 signaux ? Que pouvez-vous en conclure ? </font>

In [ ]:
sig = x1 # x1 / x2 / x3
nb_sub = 2 # nombre de sous-division du signal
nu_max = 80 # fréquence max des spectres/axe fréquentiel du spectrogramme (500 Hz si False)

In [ ]:
plot_STFT(sig,t,Te,nb_sub,numax=nu_max)

In [ ]:
plt.figure(figsize=(15,10))
from_spectrum_to_image(STFT(sig,Te,nb_sub),numax=nu_max)

On charge maintenant un signal bonus. On sait que ce signal contient du contenu fréquentiel à 10 Hz, 20 Hz, 30 Hz et 40 Hz mais on ne sait pas où sont localisés ces fréquences dans le signal.

In [ ]:
x_bonus = np.load('signaux/stft_signal_bonus.npy')

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(t,x_bonus,'b',label=r'$x_{bonus}(t)$')
plt.xlim(t.min(),t.max())
plt.legend(loc='upper right',fontsize=16)
plt.show()

<font color="blue"> **Question** Analysez le contenu fréquentiel du signal bonus en calculant son spectrogramme, et déterminez la localisation temporelle des composantes à 10 Hz, 20 Hz, 30 Hz et 40 Hz</font>

In [ ]:
# TODO